<a href="https://colab.research.google.com/github/RikiGL/Deber-Recuperacion-Informacion/blob/main/examen_Final_rikiguallichico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Descarga del dataset**


In [11]:
import kagglehub
import pandas as pd
import os

print("--- Descargando archivos del dataset ---")
# Usamos dataset_download para un control manual del archivo
path = kagglehub.dataset_download("Cornell-University/arxiv")

print(f"Archivos descargados en: {path}")

# Identificamos el archivo JSON
json_file = os.path.join(path, "arxiv-metadata-oai-snapshot.json")

print("--- Cargando muestra del dataset con Pandas ---")
cols = ['id', 'title', 'abstract', 'categories']

data = []
# Leemos el archivo por chunks
with open(json_file, 'r') as f:
    # Usamos pd.read_json con chunksize para leer solo el principio
    reader = pd.read_json(f, lines=True, chunksize=5000)
    df = next(reader) # Solo el primer bloque de 5000

# Filtramos columnas y creamos el texto combinado
df = df[cols].copy()
df['text_combined'] = df['title'] + ". " + df['abstract']

print(f"¡Éxito! Dataset cargado. Total documentos: {len(df)}")
print("Ejemplo:", df.iloc[0]['title'])

--- Descargando archivos del dataset ---
Using Colab cache for faster access to the 'arxiv' dataset.
Archivos descargados en: /kaggle/input/arxiv
--- Cargando muestra del dataset con Pandas ---
¡Éxito! Dataset cargado. Total documentos: 5000
Ejemplo: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies


 ## Preprocesamiento de Datos

In [13]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

# Configuracion inicial stopwords en ingles
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    """
    Aplica el pipeline de limpieza requerido por el examen:
    Lowercasing -> Tokenization -> Stopwords Removal -> Stemming
    """
    if not isinstance(text, str):
        return ""

    # Normalizacion
    text = text.lower()

    # Tokenizacion con regex
    tokens = re.findall(r'\b\w+\b', text)

    # Eliminación de stopwords y Stemming
    cleaned_tokens = []
    for token in tokens:
        if token not in stop_words:
            # Stemming a cada palabra que valga
            stemmed_token = stemmer.stem(token)
            cleaned_tokens.append(stemmed_token)
    return " ".join(cleaned_tokens)

print("Aplicando preprocesamiento a los documentos")
# Nueva columna 'clean_text' con el resultado
df['clean_text'] = df['text_combined'].apply(preprocess_text)

print("Preprocesamiento finalizado")

# Comparacion antes y despues
print("\n" + "="*80)
print("Comparacion de resultados")
print("="*80)

idx_ejemplo = 0 # Indice para otros ejemplos

print(f"DOCUMENTO ORIGINAL (ID: {df.iloc[idx_ejemplo]['id']}):")
print("-" * 20)
print(df.iloc[idx_ejemplo]['text_combined'][:500])
print("\n")

print(f"DOCUMENTO PREPROCESADO ")
print("-" * 20)
print(df.iloc[idx_ejemplo]['clean_text'][:500])
print("="*80)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


--- Aplicando preprocesamiento a los documentos ---
¡Preprocesamiento finalizado!

COMPARACIÓN DE RESULTADOS
DOCUMENTO ORIGINAL (ID: 704.0001):
--------------------
Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies.   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. T


DOCUMENTO PREPROCESADO (Stemming + No Stopwords):
--------------------
calcul prompt diphoton product cross section tevatron lhc energi fulli differenti calcul perturb quantum chromodynam present product massiv photon pair hadron collid next lead order perturb contribut quark antiquark gluon anti quark gluon gluon subprocess inclu

##Representación mediante Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Cargar el Modelo Pre-entrenado y usando 'all-MiniLM-L6-v2'
print("Cargando modelo SentenceTransformer")
model_embedding = SentenceTransformer('all-MiniLM-L6-v2')

print("Generando embeddings de los documentos")

# Convertimos la columna de texto limpio a una lista
corpus_sentences = df['clean_text'].tolist()

# Generamos los vectores cpn convert_to_tensor=False nos devuelve arrays de numpy para FAISS
doc_embeddings = model_embedding.encode(corpus_sentences, convert_to_tensor=False, show_progress_bar=True)

# Verificamos la forma de los embeddings
doc_embeddings = np.array(doc_embeddings) # Aseguramos formato numpy
print(f"Dimensiones de los embeddings generados: {doc_embeddings.shape}")

# Almacenamiento en Estructura de Busqueda Vectorial (FAISS)
print("Creando índice vectorial FAISS...")

d = doc_embeddings.shape[1] # Dimensión de los vectores
index = faiss.IndexFlatL2(d) # Usamos distancia L2 (Euclidiana) para busqueda exacta

index.add(doc_embeddings)# Agregamos los vectores al indice

print(f"¡Éxito! Índice FAISS creado con {index.ntotal} documentos.")

# Generacion de Embeddings para Consultas con un ejemplo

query_ejemplo = "quantum computing algorithms"
query_embedding = model_embedding.encode([query_ejemplo])

print("\n--- Ejemplo de Embedding de Consulta ---")
print(f"Consulta: '{query_ejemplo}'")
print(f"Vector generado (primeros 10 valores): {query_embedding[0][:10]}...")

## Recuperación Inicial (First-Stage Retrieval)

In [15]:
def search_initial(query_text, k=10):

    # Preprocesamiento de la consulta del paso anterior
    query_clean = preprocess_text(query_text)

    # Generar embedding de la consulta
    query_embedding = model_embedding.encode([query_clean])

    # Busqueda en el índice FAISS
    # D = Distancias (Scores)
    # I = Indices (Posicion en el DataFrame)
    D, I = index.search(np.array(query_embedding), k)

    # Recuperacion de la informacion de los documentos
    results = []
    # I[0] contiene inidices de  vecinos mas cercanos para la consulta
    for j, idx in enumerate(I[0]):
        # Verificacion indice valido
        if idx != -1 and idx < len(df):
            doc = df.iloc[idx]
            results.append({
                'rank': j + 1,
                'id': doc['id'],
                'title': doc['title'],
                'abstract': doc['abstract'],
                'text_combined': doc['text_combined'],
                'score': D[0][j]
            })

    return results

# Ejemplo
# Definimos una consulta tecnica propia de arXiv
query_prueba = "deep learning for computer vision"

print(f"Ejecutando búsqueda para: '{query_prueba}'...\n")
top_k_results = search_initial(query_prueba, k=5)

print(f"{'RANK':<5} | {'SCORE':<8} | {'TÍTULO'}")
print("-" * 80)
for res in top_k_results:
    # Mostramos Rank, Score (distancia) y Título truncado
    print(f"{res['rank']:<5} | {res['score']:.4f}   | {res['title'][:60]}...")

Ejecutando búsqueda para: 'deep learning for computer vision'...

RANK  | SCORE    | TÍTULO
--------------------------------------------------------------------------------
1     | 1.2077   | 2D Path Solutions from a Single Layer Excitable CNN Model...
2     | 1.2900   | Comparing Robustness of Pairwise and Multiclass Neural-Netwo...
3     | 1.3000   | Specialized computer algebra system for application in gener...
4     | 1.3332   | Modeling the field of laser welding melt pool by RBFNN...
5     | 1.3518   | Fast recursive filters for simulating nonlinear dynamic syst...


## Re-ranking de Resultados

In [22]:


# Cargar Modelo de Re-ranking - Cross-Encoder, dara un score de relevancia entre 2 textos
print("Cargando modelo Cross-Encoder (puede tardar unos segundos)...")
model_reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

def search_pipeline_complete(query_text, k_retrieval=20, k_rerank=5):
    # Recuperacion Inicial usando la función search_initial que creamos en el paso anterior
    initial_results = search_initial(query_text, k=k_retrieval)

    if not initial_results:
        return [], []

    # Preparacion de pares para el Cross-Encoder
    # El modelo necesita una lista de pares usando 'text_combined'
    pairs = [[query_text, doc['text_combined']] for doc in initial_results]

    # Esto devuelve un score # para cada par - mayor es mejor
    rerank_scores = model_reranker.predict(pairs)

    # Asignar nuevos scores y reordenar
    reranked_results = []
    for i, doc in enumerate(initial_results):
        # Creamos una copia para no modificar la lista original
        doc_reranked = doc.copy()
        doc_reranked['score_rerank'] = rerank_scores[i]
        reranked_results.append(doc_reranked)

    # Ordenamos descendente por el nuevo score
    reranked_results = sorted(reranked_results, key=lambda x: x['score_rerank'], reverse=True)

    # Cortamos para quedarnos solo con el top-k final
    final_results = reranked_results[:k_rerank]

    return initial_results[:k_rerank], final_results

# Ejemplo
query_demo = "machine learning for healthcare diagnosis"

print(f"Ejecutando pipeline para: '{query_demo}'...\n")
initial, final = search_pipeline_complete(query_demo, k_retrieval=30, k_rerank=5)

# Imprimir
def print_results(results, title):
    print(f"--- {title} ---")
    print(f"{'SCORE':<10} | {'TÍTULO'}")
    print("-" * 80)
    for res in results:
        sc = res.get('score_rerank', res['score'])
        print(f"{sc:.4f}     | {res['title'][:70]}...")
    print("\n")

print_results(initial, "Top 5 Originales (FAISS - Distancia)")
print_results(final, "Top 5 Después del Re-ranking (Cross-Encoder)")

Cargando modelo Cross-Encoder (puede tardar unos segundos)...
Ejecutando pipeline para: 'machine learning for healthcare diagnosis'...

--- Top 5 Originales (FAISS - Distancia) ---
SCORE      | TÍTULO
--------------------------------------------------------------------------------
1.0816     | Pr\'evention des escarres chez les parapl\'egiques : une nouvelle
  ap...
1.1023     | Artificial Tongue-Placed Tactile Biofeedback for perceptual
  suppleme...
1.1789     | An automated system for lung nodule detection in low-dose computed
  t...
1.3343     | Reduced bias nonparametric lifetime density and hazard estimation...
1.3852     | Narratives within immersive technologies...


--- Top 5 Después del Re-ranking (Cross-Encoder) ---
SCORE      | TÍTULO
--------------------------------------------------------------------------------
-5.0642     | An Adaptive Strategy for the Classification of G-Protein Coupled
  Rec...
-10.1799     | Enhancement of Noisy Planar Nuclear Medicine Images using M

 ## Simulación de Consultas

In [18]:
import pandas as pd
from IPython.display import display, HTML

# Consultas de pruebas
test_queries = [
    "dark matter detection in galaxies",
    "quantum entanglement experiments",
    "natural language processing transformers"
]

def visualize_comparison(query, initial, final):

    #Crea una visualizacion comparando Antes vs Despues
    data = []
    # Top 5 de ambos
    k_view = min(len(initial), len(final), 5)

    for i in range(k_view):
        # Datos de la Recuperacion FAISS
        init_doc = initial[i]
        init_title = init_doc['title'].replace("\n", " ") # Limpiar saltos linea
        init_score = f"{init_doc['score']:.4f}" # Distancia

        # Datos del Re-ranking (Cross-Encoder)
        final_doc = final[i]
        final_title = final_doc['title'].replace("\n", " ")
        final_score = f"{final_doc['score_rerank']:.4f}"

        data.append({
            "Rank": i + 1,
            "Inicial (FAISS)": f"{init_title[:60]}...",
            "Score Init": init_score,
            "|": "|",
            "Final (Re-ranking)": f"{final_title[:60]}...",
            "Score Final": final_score
        })

    df_compare = pd.DataFrame(data)

    print(f"\n{'='*100}")
    print(f"CONSULTA: '{query}'")
    print(f"{'='*100}")
    # Para visualizarlo como una tabla html
    display(df_compare)

# Ejecución del Bucle de Simulacion
print("Iniciando simulación de múltiples consultas...\n")

for q in test_queries:
    # Pedimos 20 candidatos iniciales y nos quedamos con los 5 mejores re-rankeados
    init_res, final_res = search_pipeline_complete(q, k_retrieval=20, k_rerank=5)
    visualize_comparison(q, init_res, final_res)

print("\nSimulación completada.")

Iniciando simulación de múltiples consultas...


CONSULTA: 'dark matter detection in galaxies'


,Rank,Inicial (FAISS),Score Init,|,Final (Re-ranking),Score Final
0,1,GLAST and Dark Matter Substructure in the Milk...,0.6868,|,Antiproton and Positron Signal Enhancement in ...,5.5319
1,2,Dark Matter Searche with GLAST...,0.7609,|,GLAST and Dark Matter Substructure in the Milk...,5.2943
2,3,"Dark matter in the Milky Way, II. the HI gas d...",0.8286,|,What it takes to measure a fundamental differe...,4.9099
3,4,Unification and Dark Matter in a Minimal Scala...,0.8371,|,Gravitational waves from galaxy encounters...,3.9714
4,5,Antiproton and Positron Signal Enhancement in ...,0.8470,|,Dark Matter Searche with GLAST...,3.8126



CONSULTA: 'quantum entanglement experiments'


,Rank,Inicial (FAISS),Score Init,|,Final (Re-ranking),Score Final
0,1,Entanglement Cost for Sequences of Arbitrary Q...,0.7114,|,Towards experimental entanglement connection w...,5.8666
1,2,Entanglement in the quantum Ising model...,0.7402,|,Entanglement and interference between differen...,5.6882
2,3,On maximal entanglement between two pairs in f...,0.7918,|,Entanglement in the quantum Ising model...,5.6422
3,4,Entanglement fidelity and measurement of entan...,0.8082,|,Entanglement and Criticality in Quantum Impuri...,5.4934
4,5,Existence of Universal Entangler...,0.8470,|,Disentanglement in a quantum critical environm...,4.9764



CONSULTA: 'natural language processing transformers'


,Rank,Inicial (FAISS),Score Init,|,Final (Re-ranking),Score Final
0,1,Pairwise comparisons of typological profiles (...,1.1132,|,Nahm transform and parabolic minimal Laplace t...,-10.7809
1,2,Entanglement fidelity and measurement of entan...,1.1302,|,Fermionic construction of tau functions and ra...,-10.8733
2,3,Connected Operators for the Totally Asymmetric...,1.1910,|,Structure of the stationary state of the asymm...,-10.9672
3,4,Quadrature formulas for the Laplace and Mellin...,1.1978,|,Two-parameter Poisson-Dirichlet measures and r...,-10.9938
4,5,Two-parameter Poisson-Dirichlet measures and r...,1.1981,|,On the generalized Freedman-Townsend model...,-11.1888



Simulación completada.


## Evaluación del Sistema

In [20]:
def search_initial(query_text, k=50): # Aumentamos default k

    query_clean = preprocess_text(query_text)
    query_embedding = model_embedding.encode([query_clean])
    D, I = index.search(np.array(query_embedding), k)

    results = []
    for j, idx in enumerate(I[0]):
        if idx != -1 and idx < len(df):
            doc = df.iloc[idx]
            results.append({
                'rank': j + 1,
                'id': doc['id'],
                'title': doc['title'],
                'abstract': doc['abstract'],
                'text_combined': doc['text_combined'],
                # --- AQUÍ ESTABA EL ERROR: AGREGAMOS LA CATEGORÍA ---
                'categories': doc['categories'],
                # ----------------------------------------------------
                'score': D[0][j]
            })
    return results

def search_pipeline_complete(query_text, k_retrieval=50, k_rerank=10):
    initial_results = search_initial(query_text, k=k_retrieval)

    if not initial_results:
        return [], []
    pairs = [[query_text, doc['text_combined']] for doc in initial_results]

    rerank_scores = model_reranker.predict(pairs)

    reranked_results = []
    for i, doc in enumerate(initial_results):
        doc_reranked = doc.copy()
        doc_reranked['score_rerank'] = rerank_scores[i]
        reranked_results.append(doc_reranked)

    reranked_results = sorted(reranked_results, key=lambda x: x['score_rerank'], reverse=True)

    return initial_results[:k_rerank], reranked_results[:k_rerank]

print("Funciones corregidas: Ahora los resultados incluyen 'categories'.")

Funciones corregidas: Ahora los resultados incluyen 'categories'.


In [21]:
import pandas as pd
import numpy as np

# Definicion de Qrels Sintéticos
# Categorias oficiales de arXiv
evaluation_topics = [
    {"query": "quantum entanglement and cryptography", "category": "quant-ph"}, # Quantum Physics
    {"query": "black hole thermodynamics", "category": "gr-qc"},             # General Relativity
    {"query": "natural language processing transformers", "category": "cs.CL"}, # Computation and Language
    {"query": "optimization algorithms for graphs", "category": "cs.DM"}     # Discrete Mathematics
]

# Pre-calculamos cuantos documentos relevantes existen en TOTAL en nuestra muestra para Recall
total_relevant_counts = {}
for topic in evaluation_topics:
    cat = topic['category']
    # Cuantos papers tienen esa categoría en nuestro en la muestra de df de 500
    count = df[df['categories'].str.contains(cat, regex=False)].shape[0]
    total_relevant_counts[cat] = count
    print(f"Categoría '{cat}': {count} documentos relevantes totales en la muestra.")

#
def calculate_metrics(results, target_category, total_relevant, k):
    # Cortamos a k
    results_k = results[:k]

    relevant_retrieved = 0
    for res in results_k:
        # Verificamos si la categoria target esta en las categorias del paper
        if target_category in res['categories']:
            relevant_retrieved += 1

    # Precision: De los k mostrados
    precision = relevant_retrieved / k if k > 0 else 0

    # Recall: De todos los relevantes que existen
    recall = relevant_retrieved / total_relevant if total_relevant > 0 else 0

    return precision, recall

# Evaluacion
print("\n" + "="*80)
print(f"{'CONSULTA':<40} | {'METRICA':<10} | {'INICIAL (FAISS)':<15} | {'FINAL (RE-RANK)':<15}")
print("="*80)

metrics_summary = {"P@10_Init": [], "P@10_Final": [], "R@10_Init": [], "R@10_Final": []}
k_eval = 10

for topic in evaluation_topics:
    q = topic['query']
    cat = topic['category']
    total_rel = total_relevant_counts[cat]

    if total_rel == 0:
        print(f"Saltando '{q}' porque no hay documentos de cat '{cat}' en la muestra.")
        continue

    # Pedimos 50 candidatos iniciales para darle margen al re-ranker de encontrar joyas
    init_res, final_res = search_pipeline_complete(q, k_retrieval=50, k_rerank=k_eval)

    # Calculamos metricas para la busqueda Inicial (FAISS)
    p_init, r_init = calculate_metrics(init_res, cat, total_rel, k_eval)

    # Calculamos metricas para el Re-ranking, tomamos el top k_eval final
    p_final, r_final = calculate_metrics(final_res, cat, total_rel, k_eval)

    # Guardamos para promedios
    metrics_summary["P@10_Init"].append(p_init)
    metrics_summary["P@10_Final"].append(p_final)
    metrics_summary["R@10_Init"].append(r_init)
    metrics_summary["R@10_Final"].append(r_final)

    # Imprimimos fila de resultados
    print(f"{q[:40]:<40} | P@{k_eval:<7} | {p_init:.4f}          | {p_final:.4f} (+{(p_final-p_init)*100:.1f}%)")
    print(f"{'':<40} | R@{k_eval:<7} | {r_init:.4f}          | {r_final:.4f}")
    print("-" * 80)

# Resumen Global
map_init = np.mean(metrics_summary["P@10_Init"])
map_final = np.mean(metrics_summary["P@10_Final"])
mar_init = np.mean(metrics_summary["R@10_Init"])
mar_final = np.mean(metrics_summary["R@10_Final"])

print("\n" + "="*80)
print("RESUMEN DE IMPACTO DEL RE-RANKING (Promedios Globales)")
print("="*80)
print(f"Precision@{k_eval} Promedio (Inicial): {map_init:.4f}")
print(f"Precision@{k_eval} Promedio (Final):   {map_final:.4f}  <-- MEJORA: {((map_final-map_init)/map_init)*100:.1f}%")
print("-" * 40)
print(f"Recall@{k_eval}    Promedio (Inicial): {mar_init:.4f}")
print(f"Recall@{k_eval}    Promedio (Final):   {mar_final:.4f}")
print("="*80)

Categoría 'quant-ph': 308 documentos relevantes totales en la muestra.
Categoría 'gr-qc': 256 documentos relevantes totales en la muestra.
Categoría 'cs.CL': 7 documentos relevantes totales en la muestra.
Categoría 'cs.DM': 17 documentos relevantes totales en la muestra.

CONSULTA                                 | METRICA    | INICIAL (FAISS) | FINAL (RE-RANK)
quantum entanglement and cryptography    | P@10      | 0.9000          | 0.9000 (+0.0%)
                                         | R@10      | 0.0292          | 0.0292
--------------------------------------------------------------------------------
black hole thermodynamics                | P@10      | 0.7000          | 0.8000 (+10.0%)
                                         | R@10      | 0.0273          | 0.0312
--------------------------------------------------------------------------------
natural language processing transformers | P@10      | 0.0000          | 0.2000 (+20.0%)
                                         | R@10  

##Análisis de Resultados

Despues de implementar un sistema Recuperación de Información de dos etapas (Retrieve & Re-rank) con el nuevo dataset

**Discusión sobre la calidad de los resultados obtenidos**

Los resultados obtenidos demuestran que el sistema es capaz de recuperar información relevante con una precisión moderada-alta dentro del corpus definido. Con una Precision@10 Final de 0.4750, el sistema entrega, en promedio, casi 5 documentos relevantes por cada 10 mostrados al usuario, lo cual es algo aceptable ya que se utilizo una muestra aleatoria limitada de documentos. La mejora del 18.8% en la precisión respecto a la fase inicial valida la efectividad de la implementacion, confirmando que la integración de modelos de lenguaje profundos (Cross-Encoders) logra mejor filtrar el ruido semántico introducido por la búsqueda vectorial simple

**Comparación entre los resultados de la recuperación inicial y el ranking final**

Al comparar las dos etapas, se observa una diferencia notable en la capacidad del sistema para priorizar contenido relevante. La recuperación inicial (FAISS) obtuvo una Precision@10 de 0.4000, funcionando eficazmente como un filtro grueso. Sin embargo, el impacto del re-ranking no solo en la precisión, sino elevo el Recall@10 de 0.0141 a 0.0865. Esto indica que el modelo de re-ranking logró identificar y "rescatar" documentos "altamente relevantes" que la búsqueda inicial había mandado a posiciones inferiores , colocandolos en el top 10